In [24]:
from typing import TYPE_CHECKING
from resinkit.flink_operation import ResultsFetchOpts
from flink_gateway_api import Client
import logging

from resinkit.flink_session import FlinkSession

# Configure the root logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logging.getLogger("httpcore.http11").setLevel(logging.INFO)
logging.getLogger('httpcore.connection').setLevel(logging.INFO)

import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops
fg_client = Client(base_url="http://localhost:8083", raise_on_unexpected_status=True)

if TYPE_CHECKING:
    from resinkit.flink_operation import FlinkOperation


sql_text = '''
-- Create Paimon catalog

DROP CATALOG IF EXISTS my_catalog;

CREATE CATALOG my_catalog WITH (
    'type'='paimon',
    'warehouse'='file:/tmp/paimon'
);

USE CATALOG my_catalog;

DROP TABLE IF EXISTS MyTable;

-- Create table with Paimon-specific options
CREATE TABLE MyTable (
    user_id BIGINT,
    item_id BIGINT,
    behavior STRING,
    dt STRING,
    PRIMARY KEY (dt, user_id) NOT ENFORCED
) PARTITIONED BY (dt) WITH (
    'bucket' = '4',
    'changelog-producer' = 'input'
);

INSERT INTO MyTable (user_id, item_id, behavior, dt) VALUES
    (1001, 5001, 'view', '2024-01-01'),
    (1001, 5002, 'like', '2024-01-01'),
    (1002, 5001, 'cart', '2024-01-01'),
    (1003, 5003, 'purchase', '2024-01-01'),
    (1001, 5004, 'view', '2024-01-02'),
    (1002, 5002, 'view', '2024-01-02'),
    (1002, 5002, 'like', '2024-01-02'),
    (1003, 5001, 'cart', '2024-01-02'),
    (1004, 5005, 'view', '2024-01-02'),
    (1001, 5003, 'purchase', '2024-01-03'),
    (1002, 5004, 'view', '2024-01-03'),
    (1003, 5002, 'like', '2024-01-03'),
    (1004, 5001, 'cart', '2024-01-03'),
    (1005, 5005, 'purchase', '2024-01-03'),
    (1001, 5001, 'view', '2024-01-04');

-- For batch query
SET 'execution.runtime-mode' = 'batch';
-- Remove the invalid orders table query
-- Instead query the created table
SELECT * FROM MyTable;
'''
dfs = []
with FlinkSession(fg_client) as session:
    for s in sql_text.split(';'):
        s = s.strip()
        if not s:
            continue
        print(f"Executing: {s}")
        with session.execute(s).sync() as operation:  # type: FlinkOperation
            dfs.append(operation.fetch(ResultsFetchOpts()).sync())
dfs

2024-12-31 11:05:02,313 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions "HTTP/1.1 200 OK"
2024-12-31 11:05:02,317 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/statements "HTTP/1.1 200 OK"
2024-12-31 11:05:02,322 - httpx - INFO - HTTP Request: GET http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/6500d5c6-3d83-45cb-812b-fe2bc209e604/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:02,322 - resinkit.session_utils - DEBUG - Fetch result: {'nextResultUri': '/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/6500d5c6-3d83-45cb-812b-fe2bc209e604/result/0?rowFormat=JSON', 'resultType': 'NOT_READY'}
2024-12-31 11:05:02,431 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/6500d5c6-3d83-45cb-812b-fe2bc209e604/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:02,432 - resinkit.session_utils - DEB

Executing: -- Create Paimon catalog

DROP CATALOG IF EXISTS my_catalog


2024-12-31 11:05:02,553 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/6500d5c6-3d83-45cb-812b-fe2bc209e604/result/1?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:02,554 - resinkit.session_utils - DEBUG - Fetch result: {'isQueryResult': False, 'resultKind': 'SUCCESS', 'resultType': 'EOS', 'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'columnInfos': [], 'data': [], 'fieldGetters': [], 'rowFormat': 'JSON'}}
2024-12-31 11:05:02,565 - httpx - INFO - HTTP Request: DELETE http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/6500d5c6-3d83-45cb-812b-fe2bc209e604/close "HTTP/1.1 200 OK"
2024-12-31 11:05:02,568 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/statements "HTTP/1.1 200 OK"
2024-12-31 11:05:02,570 - httpx - INFO - HTTP Request: GET http://

Executing: CREATE CATALOG my_catalog WITH (
    'type'='paimon',
    'warehouse'='file:/tmp/paimon'
)


2024-12-31 11:05:02,805 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/a8f127f9-a17d-40b8-91ab-23965dc8c93c/result/1?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:02,806 - resinkit.session_utils - DEBUG - Fetch result: {'isQueryResult': False, 'resultKind': 'SUCCESS', 'resultType': 'EOS', 'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'columnInfos': [], 'data': [], 'fieldGetters': [], 'rowFormat': 'JSON'}}
2024-12-31 11:05:02,807 - httpx - INFO - HTTP Request: DELETE http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/a8f127f9-a17d-40b8-91ab-23965dc8c93c/close "HTTP/1.1 200 OK"
2024-12-31 11:05:02,809 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/statements "HTTP/1.1 200 OK"
2024-12-31 11:05:02,811 - httpx - INFO - HTTP Request: GET http://

...

Executing: -- For batch query
SET 'execution.runtime-mode' = 'batch'


2024-12-31 11:05:04,082 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/de3e3c9c-7522-45ed-b7e8-cb2bc7b67792/result/1?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:04,083 - resinkit.session_utils - DEBUG - Fetch result: {'isQueryResult': False, 'resultKind': 'SUCCESS', 'resultType': 'EOS', 'results': {'columns': [{'name': 'result', 'logicalType': {'type': 'VARCHAR', 'nullable': True, 'length': 2147483647}, 'comment': None}], 'columnInfos': [], 'data': [], 'fieldGetters': [], 'rowFormat': 'JSON'}}
2024-12-31 11:05:04,085 - httpx - INFO - HTTP Request: DELETE http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/de3e3c9c-7522-45ed-b7e8-cb2bc7b67792/close "HTTP/1.1 200 OK"
2024-12-31 11:05:04,087 - httpx - INFO - HTTP Request: POST http://localhost:8083/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/statements "HTTP/1.1 200 OK"
2024-12-31 11:05:04,089 - httpx - INFO - HTTP Request: GET http://

Executing: -- Remove the invalid orders table query
-- Instead query the created table
SELECT * FROM MyTable


2024-12-31 11:05:04,301 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/d0944ef1-02ae-4577-b9ac-c9c4c23e4273/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:04,302 - resinkit.session_utils - DEBUG - Fetch result: {'nextResultUri': '/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/d0944ef1-02ae-4577-b9ac-c9c4c23e4273/result/0?rowFormat=JSON', 'resultType': 'NOT_READY'}
2024-12-31 11:05:04,406 - httpx - INFO - HTTP Request: GET http://localhost:8083/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/d0944ef1-02ae-4577-b9ac-c9c4c23e4273/result/0?rowFormat=JSON "HTTP/1.1 200 OK"
2024-12-31 11:05:04,409 - resinkit.session_utils - DEBUG - Fetch result: {'nextResultUri': '/v2/sessions/e3c1b117-ce21-4bcd-9a2b-6dfedfce801d/operations/d0944ef1-02ae-4577-b9ac-c9c4c23e4273/result/0?rowFormat=JSON', 'resultType': 'NOT_READY'}
2024-12-31 11:05:04,517 - httpx - INFO - HTTP Request: GET http://localhos

[  result
 0     OK,
   result
 0     OK,
   result
 0     OK,
   result
 0     OK,
   result
 0     OK,
                              job id
 0  1f171a4d544905597e5a6dca7232f843,
   result
 0     OK,
     user_id  item_id  behavior          dt
 0      1002     5001      cart  2024-01-01
 1      1001     5002      like  2024-01-01
 2      1003     5003  purchase  2024-01-01
 3      1001     5004      view  2024-01-02
 4      1003     5001      cart  2024-01-02
 5      1004     5005      view  2024-01-02
 6      1001     5003  purchase  2024-01-03
 7      1003     5002      like  2024-01-03
 8      1002     5002      like  2024-01-02
 9      1004     5001      cart  2024-01-03
 10     1005     5005  purchase  2024-01-03
 11     1002     5004      view  2024-01-03
 12     1001     5001      view  2024-01-04]

In [21]:
[x.shape for x in dfs]

[(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (13, 4)]

In [25]:
dfs[-1].head()

,user_id,item_id,behavior,dt
0,1002,5001,cart,2024-01-01
1,1001,5002,like,2024-01-01
2,1003,5003,purchase,2024-01-01
3,1001,5004,view,2024-01-02
4,1003,5001,cart,2024-01-02
